In [5]:
import sys
sys.path.append('/home1/04287/mxhf/HETDEX_API/')

# 01 - Finding relevant HETDEX shots

### Find all HETDEX shots, downselect by date and exposure time

In [6]:
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
from astropy.table import Table
from astropy.io import ascii

In [7]:
from hetdex_api.survey import Survey
survey = Survey('hdr1')
thdr1 = Table(survey.hdfile.root.Survey[:])

In [8]:
# read and pars shotlist
# for all repeat shots
ss = ascii.read( "/work/04287/mxhf/maverick/hetdex/cubes/shotlist_COSMOSABCD.txt", format='no_header')
ss = [np.int64(s["col1"].split('v')[0])*1000 + int( s["col1"].split('v')[1] ) for s in ss]

# get relevant rows from shotlist table
ii_ABCD = thdr1["shotid"] == 0
for s in ss:
    ii_ABCD += thdr1["shotid"] == s
#print(np.sum(ii))

# get statistics on exposure times
exptimes = []
for exptime in thdr1["exptime"][ii_ABCD]:
    exptimes.append(exptime)
exptimes = np.array( exptimes ).flatten()

min_exptime = exptimes.min() 
max_exptime = exptimes.max()
print("min_exptime = {} max_exptime = {}".format(min_exptime, max_exptime))

min_exptime = 366.2493896484375 max_exptime = 488.5973205566406


In [9]:
# for COSMOS A PCA
iiA  = thdr1["shotid"] >= 20180104007
iiA *= thdr1["shotid"] <= 20180321026

# B
iiB  = thdr1["shotid"] >= 20181101000
iiB *= thdr1["shotid"] <= 20181131999

# C
iiC  = thdr1["shotid"] >= 20181101000
iiC *= thdr1["shotid"] <= 20181131999

# D
iiD  = thdr1["shotid"] >= 20181201000
iiD *= thdr1["shotid"] <= 20190131999

# for COSMOS BCD PCA
#iiBCD  = thdr1["shotid"] >= 20181101020
#iiBCD *= thdr1["shotid"] <= 20190131999

# for GOODSN PCA
iiN  = thdr1["shotid"] >= 20180401016
iiN *= thdr1["shotid"] <= 20180631999

# for 2018 and beyond
iiN  = thdr1["shotid"] >= 20180000000
#iiN *= thdr1["shotid"] <= 20189999999

#ii = iiA + iiBCD + iiN
ii = iiN

ii *= np.array(thdr1["exptime"]).min(axis=1) >= 360.
ii *= np.array(thdr1["exptime"]).min(axis=1) <= 500.

t = Table(survey.hdfile.root.Survey[ii])

This calls the HDR1 survey class and sets each column in the survey table as an attribute to this object. This is how it looks through an astropy Table object:

In [10]:
shotlist = [ s[0] for s in Table([t["datevobs"]])]
print("{} shots.".format(len(shotlist)))

ss = ascii.read( "/work/04287/mxhf/maverick/hetdex/cubes/shotlist_GOODSN.txt", format='no_header')
print( "All repeat shots in list: ", np.all( [s[0] in shotlist for s in ss] ) )

print([s[0] in shotlist for s in ss])

1025 shots.
All repeat shots in list:  True
[True, True, True, True, True, True]


In [11]:
pcashotlist = Table( [t["datevobs"]] )
pcashotlist.write("/work/04287/mxhf/maverick/hetdex/cubes/shotlist_PCA_GOODSN.txt", format="ascii.no_header")

In [12]:
# find not yet rebinned shots
import glob
import os

ff = glob.glob("/scratch/04287/mxhf/rebin2/201*")

rebinned_shots = [os.path.basename(f) for f in ff]

missing_shots = []
for r in t:
    if not r['datevobs'] in rebinned_shots:
          missing_shots.append(r['datevobs'])
            
print("Out of {} shots in totel, {} are still missing in {}.".format(np.sum(ii),len(missing_shots),"/scratch/04287/mxhf/rebin2"))


Out of 1025 shots in totel, 14 are still missing in /scratch/04287/mxhf/rebin2.


In [13]:
missing_shots

['20190201v015',
 '20190201v016',
 '20190202v012',
 '20190203v015',
 '20190203v022',
 '20190203v023',
 '20190203v025',
 '20190205v016',
 '20190207v015',
 '20190208v028',
 '20190208v035',
 '20190209v020',
 '20190209v021',
 '20190209v027']

In [14]:
Table([missing_shots]).write("/work/04287/mxhf/maverick/hetdex/cubes/shotlist_2019_missing.txt", format="ascii.no_header")